<a href="https://colab.research.google.com/github/Boris1001/Microsleep_yawning/blob/main/Microsleep_yawning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Библиотеки

In [ ]:
from scipy.spatial import distance as dist
import numpy as np
import dlib
import cv2
from matplotlib import pyplot as plt
import time
from google.colab.patches import cv2_imshow

from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
import argparse
import imutils
import os
import re

In [ ]:
detector = dlib.get_frontal_face_detector()                                     # загружем детектор

# скачать shape_predictor_68_face_landmarks.dat (файл с весами)  можно здесь : 'https://drive.google.com/file/d/1-EblxJHiWImbDlMw5hxYR8oi40X9FxVS/view?usp=share_link'

weights = '/content/drive/MyDrive/Стажировка КАМАЗ/Dlib/shape_predictor_68_face_landmarks.dat'   # путь к перварительно скаченному файлу с весами

predictor = dlib.shape_predictor(weights)                                        # загружаем предиктор

# указываем путь к папке с фото
path = '/content/drive/MyDrive/Стажировка КАМАЗ/cropped_photos/head/'

# Устанавливанием размер обрабатывааемого изображения
width = 600 # влияет на скорость и на точность определения

# Данные

# Микросон

In [ ]:
path = '/content/drive/MyDrive/Стажировка КАМАЗ/cropped_photos/head/'           # указываем путь к папке с фото
weights = '/content/drive/MyDrive/Стажировка КАМАЗ/Dlib/shape_predictor_68_face_landmarks.dat'   # путь к перварительно скаченному файлу с весами

In [ ]:
'''

path     - путь к папке с фото
weights  - путь к перварительно скаченному файлу с весами
start    - первое фото в интрвале
finish   - последнее фото в интрвале
width    - размер фото
'''

def microsleep(path, weights, start= 0, finish = -1, width= 400):

# определяем переменные

    EYE_AR_THRESH = 0.3          # Соотношение глаз
    EYE_AR_CONSEC_FRAMES = 3     # Порог мигания

    COUNTER_flashing = 0         # Счетчик кадров для мигания
    COUNTER = 0                  # Счетчик кадров

    TOTAL = 0                    # Счетчик количества миганий


    filenames = []                                                              # определяем список с именами файлов в порядке возрастания
    result = {}                                                                 # определяем словарь для записи результата в формате: 'имя файла':  'результат определения лица'

    detector = dlib.get_frontal_face_detector()                                 # загружем детектор
    predictor = dlib.shape_predictor(weights)                                   # загружаем предиктор

    filenames = sorted(os.listdir(path), key= lambda x: int(str(re.search('\d+',x).group(0))))   # получаем список с именами файлов в порядке возрастания

# Получаем указатель на признаки лица левого и правого глаза и рта

    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

# идентифицируем точки

    FACIAL_LANDMARKS_68_IDXS = dict([
      ("mouth", (48, 68)),
      ("right_eyebrow", (17, 22)),
      ("left_eyebrow", (22, 27)),
      ("right_eye", (36, 42)),
      ("left_eye", (42, 48)),
      ("nose", (27, 36)),
      ("jaw", (0, 17))
    ])

# функция определения открытого глаза

    def eye_aspect_ratio(eye):
# Рассчитать расстояние по вертикали
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
# Рассчитать расстояние по горизонтали
        C = dist.euclidean(eye[0], eye[3])
        ear = (A + B) / (2.0 * C)
        return ear

# функция получения координат
    def shape_to_np(shape, dtype="int"):
# Создать 68 * 2
      coords = np.zeros((shape.num_parts, 2), dtype=dtype)
# Обойти каждую ключевую точку
# Получить координаты
      for i in range(0, shape.num_parts):
          coords[i] = (shape.part(i).x, shape.part(i).y)
      return coords


#    for name in filenames:                                                     # проходим по всмему списку в порядке возрастания
    for name in filenames[start:finish]:                                        # выбираем диапазон

      data = cv2.imread(path + name)                                               # считываем изображения по порядку
      if data is None:
          break
#  resize и переводим в серый
      (h, w) = data.shape[:2]

      r = width / float(w)
      dim = (width, int(h * r))
      data = cv2.resize(data, dim, interpolation=cv2.INTER_AREA)
      gray = cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)

      rects = detector(gray, 0)                                                 # Используем детектор для определения положения лица

# Зацикливание информации о положении лица и использование предиктора для получения информации о положении лица
      for rect in rects:
        shape = predictor(gray, rect)                                           # Используем предиктор для определения положения лица

        shape = face_utils.shape_to_np(shape)                                   # Преобразование информации о чертах лица в формат массива

        leftEye = shape[lStart:lEnd]                                            # Извлекаем координаты левого и правого глаза
        rightEye = shape[rStart:rEnd]

# Конструктор вычисляет значение EAR для левого и правого глаза, используя среднее значение в качестве окончательного EAR
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0

# ---------------------------------------------------------------------------------------------------------
#    смотрим что получилось, в итоговую модель вставлять не нужно
# Изображаем позицию глаз и рта

        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(data, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(data, [rightEyeHull], -1, (0, 255, 0), 1)

        # Помечаем контур лица прямоугольной рамкой
        left = rect.left()
        top = rect.top()
        right = rect.right()
        bottom = rect.bottom()
        cv2.rectangle(data, (left, top), (right, bottom), (0, 255, 0), 3)
 # ---------------------------------------------------------------------------------------------------------
        '''
                         Делаем оценку для левого и правого глаза отдельно в качестве окончательной оценки. Если она меньше порога, добавляем 1, если она меньше порога в течение трех последовательных раз, это означает, что было выполнено одно событие мигания
        '''
        # Цикл, если условие выполнено, количество миганий +1
        if ear < EYE_AR_THRESH:                                                 # проверяем соотношение глаз
            COUNTER += 1

        else:
            # Если все три раза подряд меньше порога, это означает мигание
            if COUNTER >= EYE_AR_CONSEC_FRAMES:                                 # Сравниваем с установленным порогом
                TOTAL += 1

            COUNTER = 0                                                         # Сброс счетчика кадров глаз

 # ---------------------------------------------------------------------------------------------------------
#    смотрим что получилось, в итоговую модель вставлять не нужно
        # Отображаем количества миганий
        cv2.putText(data, "Faces: {}".format(len(rects)), (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(data, "Blinks: {}".format(TOTAL), (150, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(data, "COUNTER: {}".format(COUNTER), (300, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(data, "EAR: {:.2f}".format(ear), (450, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

      cv2_imshow(data)


#        break

 # cv2.destroyAllWindows()
 # ---------------------------------------------------------------------------------------------------------

#    return result

In [ ]:
microsleep(path, weights, 50, 60, 400)

# Утомлённость

In [ ]:
'''
path     - путь к папке с фото
weights  - путь к перварительно скаченному файлу с весами
start    - первое фото в интрвале
finish   - последнее фото в интрвале
width    - размер фото
'''

def microsleep(path, weights, start= 0, finish = -1, width= 400):

# определяем переменные

    EYE_AR_THRESH = 0.3          # Соотношение глаз
    EYE_AR_CONSEC_FRAMES = 3     # Порог мигания

    MAR_THRESH = 0.5             # Соотношение рта (зевание)
    MOUTH_AR_CONSEC_FRAMES = 3   # Порог зевания

    COUNTER = 0                  # Счетчик кадров для мигания
    TOTAL = 0                    # Счетчик количества миганий

    mCOUNTER = 0                 # Счетчик кадров для зевания
    mTOTAL = 0                   # Счетчик общего количества зевания

    filenames = []                                                              # определяем список с именами файлов в порядке возрастания
    result = {}                                                                 # определяем словарь для записи результата в формате: 'имя файла':  'результат определения лица'

    detector = dlib.get_frontal_face_detector()                                 # загружем детектор
    predictor = dlib.shape_predictor(weights)                                   # загружаем предиктор

    filenames = sorted(os.listdir(path), key= lambda x: int(str(re.search('\d+',x).group(0))))   # получаем список с именами файлов в порядке возрастания


# Получаем указатель на признаки лица левого и правого глаза и рта

    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    (mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

# идентифицируем точки

    FACIAL_LANDMARKS_68_IDXS = dict([
      ("mouth", (48, 68)),
      ("right_eyebrow", (17, 22)),
      ("left_eyebrow", (22, 27)),
      ("right_eye", (36, 42)),
      ("left_eye", (42, 48)),
      ("nose", (27, 36)),
      ("jaw", (0, 17))
    ])

# функция определения открытого глаза

    def eye_aspect_ratio(eye):
# Рассчитать расстояние по вертикали
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
# Рассчитать расстояние по горизонтали
        C = dist.euclidean(eye[0], eye[3])
        ear = (A + B) / (2.0 * C)
        return ear

# функция получения координат
    def shape_to_np(shape, dtype="int"):
# Создать 68 * 2
      coords = np.zeros((shape.num_parts, 2), dtype=dtype)
# Обойти каждую ключевую точку
# Получить координаты
      for i in range(0, shape.num_parts):
          coords[i] = (shape.part(i).x, shape.part(i).y)
      return coords


#    for name in filenames:                                                     # проходим по всмему списку в порядке возрастания
    for name in filenames[start:finish]:                                        # выбираем диапазон

      data = cv2.imread(path + p)                                               # считываем изображения по порядку
      if data is None:
          break
#  resize и переводим в серый
      (h, w) = frame.shape[:2]

      r = width / float(w)
      dim = (width, int(h * r))
      data = cv2.resize(data, dim, interpolation=cv2.INTER_AREA)
      gray = cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)

      rects = detector(gray, 0)                                                 # Используем детектор для определения положения лица

# Зацикливание информации о положении лица и использование предиктора для получения информации о положении лица
    for rect in rects:
        shape = predictor(gray, rect)                                           # Используем предиктор для определения положения лица

        shape = face_utils.shape_to_np(shape)                                   # Преобразование информации о чертах лица в формат массива

        leftEye = shape[lStart:lEnd]                                            # Извлекаем координаты левого и правого глаза
        rightEye = shape[rStart:rEnd]

        mouth = shape[mStart:mEnd]                                              # Координаты рта

# Конструктор вычисляет значение EAR для левого и правого глаза, используя среднее значение в качестве окончательного EAR
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0

        mar = mouth_aspect_ratio(mouth)                                         # Определяем зевок
# ---------------------------------------------------------------------------------------------------------
#    смотрим что получилось, в итоговую модель вставлять не нужно
# Изображаем позицию глаз и рта
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        mouthHull = cv2.convexHull(mouth)
        cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)

        # Помечаем контур лица прямоугольной рамкой
        left = rect.left()
        top = rect.top()
        right = rect.right()
        bottom = rect.bottom()
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 3)
 # ---------------------------------------------------------------------------------------------------------
        '''
                         Делаем оценку для левого и правого глаза отдельно в качестве окончательной оценки. Если она меньше порога, добавляем 1, если она меньше порога в течение трех последовательных раз, это означает, что было выполнено одно событие мигания
        '''
        # Цикл, если условие выполнено, количество миганий +1
        if ear < EYE_AR_THRESH:                        # проверяем соотношение глаз
            COUNTER += 1

        else:
            # Если все три раза подряд меньше порога, это означает мигание
            if COUNTER >= EYE_AR_CONSEC_FRAMES:# Сравниваем с установленным порогом
                TOTAL += 1

            COUNTER = 0                                # Сброс счетчика кадров глаз

 # ---------------------------------------------------------------------------------------------------------
#    смотрим что получилось, в итоговую модель вставлять не нужно
        # Отображаем количества миганий
        cv2.putText(frame, "Faces: {}".format(len(rects)), (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "Blinks: {}".format(TOTAL), (150, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "COUNTER: {}".format(COUNTER), (300, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (450, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
  # ---------------------------------------------------------------------------------------------------------

        '''
                         Делаем оценку открытого рта, если она меньше порога, добаляем 1, если он меньше порога три раза подряд, это означает, что зевок сделан, и тот же зевок составляет около 3 кадров.
        '''

        # Цикл, если условие выполнено, количество зеваний +1
        if mar > MAR_THRESH:                           # проверяем соотношение рта (определяем зевание)
            mCOUNTER += 1
            cv2.putText(frame, "Yawning!", (10, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            # Если оно меньше порога 3 раза подряд, это означает зевок
            if mCOUNTER >= MOUTH_AR_CONSEC_FRAMES:     # Сравниваем с установленным порогом
                mTOTAL += 1
            # Сброс счетчика кадров рта
            mCOUNTER = 0

# ---------------------------------------------------------------------------------------------------------
#    смотрим что получилось, в итоговую модель вставлять не нужно
        cv2.putText(frame, "Yawning: {}".format(mTOTAL), (150, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "mCOUNTER: {}".format(mCOUNTER), (300, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "MAR: {:.2f}".format(mar), (480, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Операция рисования, 68 идентификация характерных точек
        for (x, y) in shape:
            cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)
# ---------------------------------------------------------------------------------------------------------


    # Определяем усталость
    if TOTAL >= 50 or mTOTAL>=15:
 # ---------------------------------------------------------------------------------------------------------
#    смотрим что получилось, в итоговую модель вставлять не нужно
        cv2.putText(frame, "SLEEP!!!", (100, 200),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    # Нажмите q для выхода
    # cv2.putText(frame, "Press 'q': Quit", (20, 500),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (84, 255, 159), 2)
    # Показ окна с помощью opencv
    cv2_imshow(frame)


    # if the `q` key was pressed, break from the loop
#   if cv2.waitKey(1) & 0xFF == ord('q'):
#        break

 # cv2.destroyAllWindows()
 # ---------------------------------------------------------------------------------------------------------

    return result

In [ ]:
def microsleep(path, width):

  EYE_AR_THRESH = 0.3          # Соотношение глаз
  EYE_AR_CONSEC_FRAMES = 3     # Порог мигания

  MAR_THRESH = 0.5             # Соотношение рта (зевание)
  MOUTH_AR_CONSEC_FRAMES = 3   # Порог зевания

  COUNTER = 0                  # Счетчик кадров для мигания
  TOTAL = 0                    # Счетчик количества миганий

  mCOUNTER = 0                 # Счетчик кадров для зевания
  mTOTAL = 0                   # Счетчик общего количества зевания


  # Получаем указатель на признаки лица левого и правого глаза и рта
  (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
  (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
  (mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

  # идентифицируем точки
  FACIAL_LANDMARKS_68_IDXS = dict([
    ("mouth", (48, 68)),
    ("right_eyebrow", (17, 22)),
    ("left_eyebrow", (22, 27)),
    ("right_eye", (36, 42)),
    ("left_eye", (42, 48)),
    ("nose", (27, 36)),
    ("jaw", (0, 17))
  ])

  # функция определения открытого глаза
  def eye_aspect_ratio(eye):
    # Рассчитать расстояние по вертикали
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    # Рассчитать расстояние по горизонтали
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear
  # функция получения координат
  def shape_to_np(shape, dtype="int"):
    # Создать 68 * 2
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)
    # Обойти каждую ключевую точку
    # Получить координаты
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)
    return coords


  for p in os.listdir(path):
    startTime = time.time()
    frame = cv2.imread(path + p)
    if frame is None:
        break

    (h, w) = frame.shape[:2]

    r = width / float(w)
    dim = (width, int(h * r))
    frame = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Используем детектор (серый, 0) для определения положения лица
    rects = detector(gray, 0)

    # Зацикливание информации о положении лица и использование предиктора (серый, прямоугольный) для получения информации о положении лица
    for rect in rects:
        shape = predictor(gray, rect)

        # Преобразование информации о чертах лица в формат массива
        shape = face_utils.shape_to_np(shape)

        # Извлекаем координаты левого и правого глаза
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]

        # Координаты рта
        mouth = shape[mStart:mEnd]

        # Конструктор вычисляет значение EAR для левого и правого глаз, используя среднее значение в качестве окончательного EAR
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0

        # Определяем зевок
        mar = mouth_aspect_ratio(mouth)

        # Изображаем позицию глаз и рта

        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        mouthHull = cv2.convexHull(mouth)
        cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)

        # Помечаем контур лица прямоугольной рамкой
        left = rect.left()
        top = rect.top()
        right = rect.right()
        bottom = rect.bottom()
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 3)

        '''
                         Делаем оценку для левого и правого глаза отдельно в качестве окончательной оценки. Если она меньше порога, добавляем 1, если она меньше порога в течение трех последовательных раз, это означает, что было выполнено одно событие мигания
        '''
        # Цикл, если условие выполнено, количество миганий +1
        if ear < EYE_AR_THRESH:                        # проверяем соотношение глаз
            COUNTER += 1

        else:
            # Если все три раза подряд меньше порога, это означает мигание
            if COUNTER >= EYE_AR_CONSEC_FRAMES:# Сравниваем с установленным порогом
                TOTAL += 1

            COUNTER = 0                                # Сброс счетчика кадров глаз

        # Отображаем количества миганий
        cv2.putText(frame, "Faces: {}".format(len(rects)), (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "Blinks: {}".format(TOTAL), (150, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "COUNTER: {}".format(COUNTER), (300, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (450, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        '''
                         Делаем оценку открытого рта, если она меньше порога, добаляем 1, если он меньше порога три раза подряд, это означает, что зевок сделан, и тот же зевок составляет около 3 кадров.
        '''
        # Цикл, если условие выполнено, количество зеваний +1
        if mar > MAR_THRESH:                           # проверяем соотношение рта (определяем зевание)
            mCOUNTER += 1
            cv2.putText(frame, "Yawning!", (10, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            # Если оно меньше порога 3 раза подряд, это означает зевок
            if mCOUNTER >= MOUTH_AR_CONSEC_FRAMES:     # Сравниваем с установленным порогом
                mTOTAL += 1
            # Сброс счетчика кадров рта
            mCOUNTER = 0
        cv2.putText(frame, "Yawning: {}".format(mTOTAL), (150, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "mCOUNTER: {}".format(mCOUNTER), (300, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "MAR: {:.2f}".format(mar), (480, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Операция рисования, 68 идентификация характерных точек
        for (x, y) in shape:
            cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)


    # Определяем усталость
    if TOTAL >= 50 or mTOTAL>=15:

        cv2.putText(frame, "SLEEP!!!", (100, 200),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    # Нажмите q для выхода
    # cv2.putText(frame, "Press 'q': Quit", (20, 500),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (84, 255, 159), 2)
    # Показ окна с помощью opencv
    cv2_imshow(frame)

    # if the `q` key was pressed, break from the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

  cv2.destroyAllWindows()

# Отвлечение взгляда

In [ ]:
path = '/content/drive/MyDrive/Стажировка КАМАЗ/cropped_photos/head/'           # путь к папке с фото

In [ ]:
'''
path   - путь к папке с фото
start  - первое фото в интрвале
finish - последнее фото в интрвале
width  - размер фото
'''

def distraction(path, start= 0, finish = -1, width= 400):

    filenames = []                                                              # определяем список с именами файлов в порядке возрастания
    result = {}                                                                 # определяем словарь для записи результата в формате: 'имя файла':  'результат определения лица'

    detector = dlib.get_frontal_face_detector()                                 # загружем детектор

    filenames = sorted(os.listdir(path), key= lambda x: int(str(re.search('\d+',x).group(0))))   # получаем список с именами файлов в порядке возрастания


#    for name in filenames:                                                     # проходим по всмему списку в порядке возрастания
    for name in filenames[start:finish]:                                        # выбираем диапазон

        data = cv2.imread(path + name)                                          # считываем изображения по порядку

        if data is None:
            break

        (h, w) = data.shape[:2]
        r = width / float(w)
        dim = (width, int(h * r))
        data = cv2.resize(data, dim, interpolation=cv2.INTER_AREA)              # меняем размер фото

        face = detector(data,0)                                                 # получаем координаты лица

        if face:
          result[name] = result.get(name, True)                                 # если координаты есть - добавляем по ключу(имя изображение) значение (True)
        else:
          result[name] = result.get(name, False)                                # если координат нет - добавляем по ключу(имя изображение) значение (False)

# ---------------------------------------------------------------------------------------------------------
#    смотрим что получилось, в итоговую модель вставлять не нужно

        for rect in face:
          left = rect.left()
          top = rect.top()
          right = rect.right()
          bottom = rect.bottom()
          cv2.rectangle(data, (left, top), (right, bottom), (0, 255, 0), 3)

        cv2_imshow(data)

# ---------------------------------------------------------------------------------------------------------

    return result

In [ ]:
res = distraction(path, 135, 170, 400)